In [18]:
from NseUtility import NseUtils
import time
from datetime import datetime
import pandas as pd
import nsepython
import pdb
import pprint

In [7]:
# Initialize NseUtility
nse = NseUtils()

In [11]:
#!/usr/bin/env python3
"""
NSE Sectoral Indices Real-Time Performance Tracker
Using NseUtility library - Handles DataFrame response
Fixed: UTF-8 encoding for emoji support on Windows
"""

# Check current time and market status
now = datetime.now()
print(f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Note: NSE market hours are 9:15 AM - 3:30 PM IST\n")

SECTORAL_INDICES = [
    "NIFTY INDIA CONSUMPTION",
    "NIFTY FINANCIAL SERVICES",
    "NIFTY PHARMA",
    "NIFTY AUTO",
    "NIFTY PRIVATE BANK",
    "NIFTY COMMODITIES",
    "NIFTY MNC",
    "NIFTY BANK",
    "NIFTY METAL",
    "NIFTY SERVICES SECTOR",
    "NIFTY PSE",
    "NIFTY INFRASTRUCTURE",
    "NIFTY SMALLCAP 100",
    "NIFTY TATA GROUP 25% CAP",
    "NIFTY MIDCAP 50",
    "NIFTY PSU BANK",
    "NIFTY ENERGY",
    "NIFTY IT",
    "NIFTY FMCG",
    "NIFTY CPSE",
    "NIFTY50 USD",
    "NIFTY OIL & GAS",
    "NIFTY NEXT 50",
    "NIFTY 50",
    "NIFTY GROWTH SECTORS 15",
    "NIFTY 100",
    "NIFTY 500",
    "NIFTY MIDCAP 100",
    "NIFTY FINANCIAL SERVICES 25/50",
    "NIFTY REALTY",
    "NIFTY HEALTHCARE INDEX",
    "NIFTY CONSUMER DURABLES"
]

def safe_float_convert(value, default=0.0):
    """Safely convert value to float"""
    if value is None or value == "" or value == "-" or pd.isna(value):
        return default
    try:
        return float(str(value).replace(",", ""))
    except (ValueError, AttributeError, TypeError):
        return default

def fetch_index_data(index_name):
    """
    Fetch index data using NseUtility
    Returns: DataFrame from get_index_details()
    """
    try:
        # NseUtility.get_index_details() returns DataFrame
        df = nse.get_index_details(index_name)

        if df is None:
            return None

        # Check if it's a DataFrame
        if isinstance(df, pd.DataFrame):
            if df.empty:
                return None
            return df

        # If it returns a dict, convert to DataFrame
        elif isinstance(df, dict):
            return pd.DataFrame([df])

        return None

    except Exception as e:
        # Silently handle errors to keep output clean
        return None

def extract_index_info(df_result, index_name):
    """
    Extract relevant information from DataFrame result
    Handles various column name formats
    """
    try:
        if df_result is None or df_result.empty:
            return None

        # Get first row if multiple rows
        if len(df_result) > 0:
            row = df_result.iloc[0]
        else:
            return None

        # Helper function to try multiple column names
        def get_value(row, *keys):
            for key in keys:
                if key in row.index:
                    val = row[key]
                    if pd.notna(val):
                        return val
            return None

        # Extract values with multiple possible column names
        ltp = safe_float_convert(get_value(row, 'last', 'lastPrice', 'ltp', 'Last', 'close', 'Close'))
        prev_close = safe_float_convert(get_value(row, 'previousClose', 'prevClose', 'PreviousClose', 'Prev Close'))
        open_price = safe_float_convert(get_value(row, 'open', 'Open', 'openPrice'))
        high_price = safe_float_convert(get_value(row, 'dayHigh', 'high', 'High', 'highPrice'))
        low_price = safe_float_convert(get_value(row, 'dayLow', 'low', 'Low', 'lowPrice'))
        change = safe_float_convert(get_value(row, 'change', 'Change', 'priceChange'))
        change_pct = safe_float_convert(get_value(row, 'pChange', 'percChange', 'percentChange', 'Change%', 'pctChange'))

        # Calculate change if not provided
        if change == 0.0 and ltp > 0 and prev_close > 0:
            change = ltp - prev_close

        # Calculate percentage change if not provided
        if change_pct == 0.0 and change != 0.0 and prev_close > 0:
            change_pct = (change / prev_close) * 100

        # Get timestamp
        timestamp = get_value(row, 'timestamp', 'lastUpdateTime', 'timeVal', 'Time', 'LastUpdate') or "N/A"

        # Get sector name
        sector_name = get_value(row, 'name', 'indexName', 'symbol', 'Symbol', 'Index') or index_name

        return {
            "Sector": sector_name,
            "LTP": ltp,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Prev Close": prev_close,
            "Change": change,
            "Change %": change_pct,
            "Updated": timestamp
        }

    except Exception as e:
        return None

def main():
    sector_data = []

    print("Fetching sectoral indices data...\n")

    for idx, sector in enumerate(SECTORAL_INDICES, 1):
        print(f"[{idx:2d}/{len(SECTORAL_INDICES)}] {sector:<35}", end=" ")

        try:
            # Fetch DataFrame
            df_result = fetch_index_data(sector)

            if df_result is None:
                print("❌ No data")
                continue

            # Extract information from DataFrame
            index_info = extract_index_info(df_result, sector)

            if index_info is None:
                print("❌ Extraction failed")
                continue

            # Validate data
            if index_info["LTP"] == 0 and index_info["Prev Close"] == 0:
                print("❌ Invalid data")
                continue

            sector_data.append(index_info)
            print(f"✅ {index_info['Change %']:>6.2f}%")

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            continue

        # Small delay to avoid overwhelming the server
        time.sleep(0.3)

    if not sector_data:
        print("\n⚠️ No data fetched. Check your internet connection or try again later.")
        return None

    # Create DataFrame and sort by performance
    df = pd.DataFrame(sector_data).sort_values("Change %", ascending=False)

    # Display results
    print("\n" + "="*130)
    print("=== REAL-TIME SECTORAL PERFORMANCE ===")
    print("="*130)

    # Format numeric columns for display
    display_df = df.copy()
    display_df['LTP'] = display_df['LTP'].apply(lambda x: f"{x:,.2f}")
    display_df['Open'] = display_df['Open'].apply(lambda x: f"{x:,.2f}")
    display_df['High'] = display_df['High'].apply(lambda x: f"{x:,.2f}")
    display_df['Low'] = display_df['Low'].apply(lambda x: f"{x:,.2f}")
    display_df['Prev Close'] = display_df['Prev Close'].apply(lambda x: f"{x:,.2f}")
    display_df['Change'] = display_df['Change'].apply(lambda x: f"{x:+,.2f}")
    display_df['Change %'] = display_df['Change %'].apply(lambda x: f"{x:+.2f}%")

    print(display_df.to_string(index=False))

    # Top Gainers
    print("\n" + "="*130)
    print("=== TOP 5 GAINERS ===")
    print("="*130)
    top_gainers = df.head(5)
    for idx, row in top_gainers.iterrows():
        print(f"🚀 {row['Sector']:<40} {row['Change %']:>6.2f}%  (LTP: {row['LTP']:>10,.2f}, Change: {row['Change']:>+8,.2f})")

    # Top Losers
    print("\n" + "="*130)
    print("=== TOP 5 LOSERS ===")
    print("="*130)
    top_losers = df.tail(5).iloc[::-1]
    for idx, row in top_losers.iterrows():
        print(f"📉 {row['Sector']:<40} {row['Change %']:>6.2f}%  (LTP: {row['LTP']:>10,.2f}, Change: {row['Change']:>+8,.2f})")

    # Market Summary
    print("\n" + "="*130)
    print("=== MARKET SUMMARY ===")
    print("="*130)
    gainers = len(df[df['Change %'] > 0])
    losers = len(df[df['Change %'] < 0])
    unchanged = len(df[df['Change %'] == 0])

    print(f"Total sectors fetched: {len(df)}")
    print(f"Gainers:   {gainers:2d} ({gainers/len(df)*100:5.1f}%) 📈")
    print(f"Losers:    {losers:2d} ({losers/len(df)*100:5.1f}%) 📉")
    print(f"Unchanged: {unchanged:2d} ({unchanged/len(df)*100:5.1f}%) ➡️")

    if len(df) > 0:
        avg_change = df['Change %'].mean()
        print(f"\nAverage Change: {avg_change:+.2f}%")
        print(f"Best Performer: {df.iloc[0]['Sector']} ({df.iloc[0]['Change %']:+.2f}%)")
        print(f"Worst Performer: {df.iloc[-1]['Sector']} ({df.iloc[-1]['Change %']:+.2f}%)")

    # Save to CSV with timestamp
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    filename = f'nse_sectoral_{timestamp}.csv'
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"\n✅ Data saved to '{filename}'")

    # Generate HTML report
    html_filename = f'nse_sectoral_{timestamp}.html'

    html_content = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>NSE Sectoral Indices - {now.strftime('%Y-%m-%d %H:%M:%S')}</title>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
        }}
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 10px;
            padding: 30px;
            box-shadow: 0 10px 40px rgba(0,0,0,0.3);
        }}
        h1 {{
            color: #2d3748;
            border-bottom: 4px solid #667eea;
            padding-bottom: 15px;
            margin-top: 0;
        }}
        .header-info {{
            background: #f7fafc;
            padding: 15px;
            border-radius: 5px;
            margin: 20px 0;
            border-left: 4px solid #667eea;
        }}
        .stats {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            margin: 30px 0;
        }}
        .stat-box {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 25px;
            border-radius: 10px;
            text-align: center;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            transition: transform 0.3s ease;
        }}
        .stat-box:hover {{
            transform: translateY(-5px);
        }}
        .stat-label {{
            font-size: 14px;
            opacity: 0.9;
            margin-bottom: 10px;
        }}
        .stat-value {{
            font-size: 32px;
            font-weight: bold;
        }}
        .stat-sub {{
            font-size: 14px;
            opacity: 0.8;
            margin-top: 5px;
        }}
        .gainer-box {{
            background: linear-gradient(135deg, #48bb78 0%, #38a169 100%);
        }}
        .loser-box {{
            background: linear-gradient(135deg, #f56565 0%, #e53e3e 100%);
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }}
        th {{
            background: linear-gradient(135deg, #2d3748 0%, #1a202c 100%);
            color: white;
            padding: 15px;
            text-align: right;
            font-weight: 600;
            position: sticky;
            top: 0;
        }}
        th.sector {{
            text-align: left;
        }}
        td {{
            padding: 12px 15px;
            border-bottom: 1px solid #e2e8f0;
            text-align: right;
        }}
        td.sector {{
            text-align: left;
            font-weight: 600;
            color: #2d3748;
        }}
        tr:hover {{
            background-color: #f7fafc;
        }}
        .positive {{
            color: #38a169;
            font-weight: bold;
        }}
        .negative {{
            color: #e53e3e;
            font-weight: bold;
        }}
        .footer {{
            margin-top: 30px;
            padding: 20px;
            background: #f7fafc;
            border-radius: 5px;
            text-align: center;
            color: #718096;
            font-size: 14px;
        }}
        .emoji {{
            font-size: 24px;
            margin-right: 10px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1><span class="emoji">📊</span>NSE Sectoral Indices Performance</h1>

        <div class="header-info">
            <strong>Generated:</strong> {now.strftime('%Y-%m-%d %H:%M:%S IST')} |
            <strong>Library:</strong> NseUtility |
            <strong>Total Indices:</strong> {len(df)}
        </div>

        <div class="stats">
            <div class="stat-box">
                <div class="stat-label">Total Sectors</div>
                <div class="stat-value">{len(df)}</div>
            </div>
            <div class="stat-box gainer-box">
                <div class="stat-label">Gainers</div>
                <div class="stat-value">{gainers}</div>
                <div class="stat-sub">{gainers/len(df)*100:.1f}% of total</div>
            </div>
            <div class="stat-box loser-box">
                <div class="stat-label">Losers</div>
                <div class="stat-value">{losers}</div>
                <div class="stat-sub">{losers/len(df)*100:.1f}% of total</div>
            </div>
            <div class="stat-box">
                <div class="stat-label">Avg Change</div>
                <div class="stat-value">{df['Change %'].mean():+.2f}%</div>
            </div>
        </div>

        <table>
            <thead>
                <tr>
                    <th class="sector">Sector</th>
                    <th>LTP</th>
                    <th>Open</th>
                    <th>High</th>
                    <th>Low</th>
                    <th>Prev Close</th>
                    <th>Change</th>
                    <th>Change %</th>
                </tr>
            </thead>
            <tbody>
"""

    for idx, row in df.iterrows():
        change_class = "positive" if row['Change %'] > 0 else "negative" if row['Change %'] < 0 else ""
        html_content += f"""                <tr>
                    <td class="sector">{row['Sector']}</td>
                    <td>{row['LTP']:,.2f}</td>
                    <td>{row['Open']:,.2f}</td>
                    <td>{row['High']:,.2f}</td>
                    <td>{row['Low']:,.2f}</td>
                    <td>{row['Prev Close']:,.2f}</td>
                    <td class="{change_class}">{row['Change']:+,.2f}</td>
                    <td class="{change_class}">{row['Change %']:+.2f}%</td>
                </tr>
"""

    html_content += f"""            </tbody>
        </table>

        <div class="footer">
            <p><strong>Market Hours:</strong> 9:15 AM - 3:30 PM IST</p>
            <p>Data sourced from NSE through NseUtility. For informational purposes only.</p>
            <p>Best Performer: <strong>{df.iloc[0]['Sector']}</strong> ({df.iloc[0]['Change %']:+.2f}%) |
               Worst Performer: <strong>{df.iloc[-1]['Sector']}</strong> ({df.iloc[-1]['Change %']:+.2f}%)</p>
        </div>
    </div>
</body>
</html>
"""

    # CRITICAL FIX: Use UTF-8 encoding for emoji support
    with open(html_filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    print(f"✅ HTML report saved to '{html_filename}'")

    return df

if __name__ == "__main__":
    try:
        print("="*130)
        print("NSE SECTORAL INDICES TRACKER - NseUtility (DataFrame Mode)")
        print("="*130 + "\n")

        df = main()

        if df is not None:
            print("\n" + "="*130)
            print("✅ Script execution completed successfully!")
            print("="*130)
        else:
            print("\n⚠️ No data could be fetched. Please try again later.")

    except KeyboardInterrupt:
        print("\n\n⚠️ Script interrupted by user")
    except Exception as e:
        print(f"\n\n❌ Unexpected Error: {str(e)}")
        import traceback
        traceback.print_exc()

Current time: 2025-09-30 12:23:43
Note: NSE market hours are 9:15 AM - 3:30 PM IST

NSE SECTORAL INDICES TRACKER - NseUtility (DataFrame Mode)

Fetching sectoral indices data...

[ 1/32] NIFTY INDIA CONSUMPTION             ✅  -0.15%
[ 2/32] NIFTY FINANCIAL SERVICES            ✅   0.14%
[ 3/32] NIFTY PHARMA                        ✅   0.10%
[ 4/32] NIFTY AUTO                          ✅  -0.04%
[ 5/32] NIFTY PRIVATE BANK                  ✅   0.21%
[ 6/32] NIFTY COMMODITIES                   ✅   0.42%
[ 7/32] NIFTY MNC                           ✅   0.03%
[ 8/32] NIFTY BANK                          ✅   0.48%
[ 9/32] NIFTY METAL                         ✅   1.43%
[10/32] NIFTY SERVICES SECTOR               ✅   0.14%
[11/32] NIFTY PSE                           ✅   0.33%
[12/32] NIFTY INFRASTRUCTURE                ✅  -0.28%
[13/32] NIFTY SMALLCAP 100                  ✅  -0.09%
[14/32] NIFTY TATA GROUP 25% CAP            ❌ No data
[15/32] NIFTY MIDCAP 50                     ✅  -0.14%
[16/32] NIF

In [ ]:
L = nsepython.nse_get_fno_lot_sizes(symbol='KPITTECH')

In [13]:
    #!/usr/bin/env python3
    """
    NSE Sectoral Indices with Top Stock Performers - Smart Extraction
    Automatically detects DataFrame structure and extracts stocks directly
    No separate API calls needed!
    """
    # Check current time and market status
    now = datetime.now()
    print(f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Note: NSE market hours are 9:15 AM - 3:30 PM IST\n")

    SECTORAL_INDICES = [
        "NIFTY INDIA CONSUMPTION",
        "NIFTY FINANCIAL SERVICES",
        "NIFTY PHARMA",
        "NIFTY AUTO",
        "NIFTY PRIVATE BANK",
        "NIFTY COMMODITIES",
        "NIFTY MNC",
        "NIFTY BANK",
        "NIFTY METAL",
        "NIFTY SERVICES SECTOR",
        "NIFTY PSE",
        "NIFTY INFRASTRUCTURE",
        "NIFTY SMALLCAP 100",
        "NIFTY MIDCAP 50",
        "NIFTY PSU BANK",
        "NIFTY ENERGY",
        "NIFTY IT",
        "NIFTY FMCG",
        "NIFTY CPSE",
        "NIFTY OIL & GAS",
        "NIFTY NEXT 50",
        "NIFTY 50",
        "NIFTY GROWTH SECTORS 15",
        "NIFTY 100",
        "NIFTY 500",
        "NIFTY MIDCAP 100",
        "NIFTY FINANCIAL SERVICES 25/50",
        "NIFTY REALTY",
        "NIFTY HEALTHCARE INDEX",
        "NIFTY CONSUMER DURABLES"
    ]

    def safe_float_convert(value, default=0.0):
        """Safely convert value to float"""
        if value is None or value == "" or value == "-" or pd.isna(value):
            return default
        try:
            return float(str(value).replace(",", ""))
        except (ValueError, AttributeError, TypeError):
            return default

    def smart_extract_stocks_from_df(df, index_name, verbose=False):
        """
        Intelligently extract stocks from DataFrame returned by get_index_details()
        Handles multiple possible DataFrame structures

        Returns: (index_info, stocks_list)
        """
        try:
            if df is None or df.empty:
                return None, None

            if verbose:
                print(f"\n      Analyzing DataFrame structure...")
                print(f"      Shape: {df.shape}")
                print(f"      Columns: {list(df.columns)}")

            # Initialize variables
            index_info = None
            stocks = []

            # Possible column name variations
            SYMBOL_COLS = ['symbol', 'Symbol', 'SYMBOL', 'stock', 'Stock', 'ticker', 'Ticker']
            NAME_COLS = ['name', 'Name', 'companyName', 'Company', 'company', 'stockName']
            LTP_COLS = ['last', 'lastPrice', 'ltp', 'LTP', 'close', 'Close', 'price', 'Price']
            CHANGE_PCT_COLS = ['pChange', 'percChange', 'percentChange', 'Change%', 'change%', 'pctChange', 'changePercent']

            # Helper function to find matching column
            def find_column(df, possible_names):
                for name in possible_names:
                    if name in df.columns:
                        return name
                return None

            # Find relevant columns
            symbol_col = find_column(df, SYMBOL_COLS)
            name_col = find_column(df, NAME_COLS)
            ltp_col = find_column(df, LTP_COLS)
            change_col = find_column(df, CHANGE_PCT_COLS)

            if verbose:
                print(f"      Detected columns: symbol={symbol_col}, name={name_col}, ltp={ltp_col}, change%={change_col}")

            # STRATEGY 1: Check if DataFrame has stock symbols
            # If it has symbols, treat all rows (or rows 1+) as stocks
            if symbol_col:
                # Determine starting row
                # If first row has no symbol or looks like index summary, start from row 1
                first_symbol = df.iloc[0][symbol_col] if len(df) > 0 else None

                # Check if first row is index or stock
                start_idx = 0
                if pd.isna(first_symbol) or first_symbol == "" or first_symbol == index_name:
                    start_idx = 1

                    # Extract index info from first row if present
                    if len(df) > 0 and ltp_col and change_col:
                        index_ltp = safe_float_convert(df.iloc[0][ltp_col])
                        index_change = safe_float_convert(df.iloc[0][change_col])

                        if index_ltp > 0 or index_change != 0:
                            index_info = {
                                "Sector": index_name,
                                "LTP": index_ltp,
                                "Change %": index_change
                            }

                # Extract stocks
                for idx in range(start_idx, len(df)):
                    row = df.iloc[idx]

                    symbol = row[symbol_col] if symbol_col else None
                    if pd.isna(symbol) or symbol == "" or symbol == index_name:
                        continue

                    company = row[name_col] if name_col else symbol
                    stock_ltp = safe_float_convert(row[ltp_col]) if ltp_col else 0.0
                    stock_change = safe_float_convert(row[change_col]) if change_col else 0.0

                    # Only add if we have valid data
                    if symbol and (stock_ltp > 0 or stock_change != 0):
                        stocks.append({
                            'Symbol': str(symbol).strip(),
                            'Company': str(company).strip() if company else str(symbol).strip(),
                            'LTP': stock_ltp,
                            'Change %': stock_change
                        })

            # STRATEGY 2: No symbol column - might be just index data
            else:
                # Just extract index info
                if len(df) > 0 and ltp_col and change_col:
                    index_ltp = safe_float_convert(df.iloc[0][ltp_col])
                    index_change = safe_float_convert(df.iloc[0][change_col])

                    index_info = {
                        "Sector": index_name,
                        "LTP": index_ltp,
                        "Change %": index_change
                    }

            # If we didn't get index info, try to calculate from first row
            if not index_info and len(df) > 0:
                if ltp_col and change_col:
                    index_info = {
                        "Sector": index_name,
                        "LTP": safe_float_convert(df.iloc[0][ltp_col]),
                        "Change %": safe_float_convert(df.iloc[0][change_col])
                    }

            return index_info, stocks if stocks else None

        except Exception as e:
            if verbose:
                print(f"      Error in extraction: {str(e)}")
            return None, None

    def main():
        all_indices_data = []
        all_stocks_by_index = {}

        print("="*130)
        print("PHASE 1: FETCHING SECTORAL INDICES DATA WITH CONSTITUENT STOCKS")
        print("="*130 + "\n")

        # Test first index with verbose output to understand structure
        first_run = True

        for idx, sector in enumerate(SECTORAL_INDICES, 1):
            print(f"[{idx:2d}/{len(SECTORAL_INDICES)}] {sector:<35}", end=" ")

            try:
                # Fetch DataFrame
                df = nse.get_index_details(sector)

                if df is None or df.empty:
                    print("❌ No data")
                    continue

                # Extract index info and stocks (verbose for first run)
                index_info, stocks = smart_extract_stocks_from_df(df, sector, verbose=first_run)
                first_run = False

                if index_info is None:
                    print("❌ Extraction failed")
                    continue

                # Store index info
                all_indices_data.append(index_info)

                # Store stocks if available
                if stocks and len(stocks) > 0:
                    all_stocks_by_index[sector] = stocks
                    print(f"✅ {index_info['Change %']:>6.2f}% | {len(stocks)} stocks")
                else:
                    print(f"✅ {index_info['Change %']:>6.2f}% | No stocks")

            except Exception as e:
                print(f"❌ Error: {str(e)}")
                continue

            time.sleep(0.3)

        if not all_indices_data:
            print("\n⚠️ No data fetched.")
            return None

        # Create DataFrame and sort
        df_indices = pd.DataFrame(all_indices_data).sort_values("Change %", ascending=False)

        print("\n" + "="*130)
        print("=== SECTORAL PERFORMANCE SUMMARY ===")
        print("="*130)
        print(f"Total indices fetched: {len(df_indices)}")
        print(f"Indices with stock data: {len(all_stocks_by_index)}")
        print(f"Gainers: {len(df_indices[df_indices['Change %'] > 0])} ({len(df_indices[df_indices['Change %'] > 0])/len(df_indices)*100:.1f}%)")
        print(f"Losers: {len(df_indices[df_indices['Change %'] < 0])} ({len(df_indices[df_indices['Change %'] < 0])/len(df_indices)*100:.1f}%)")

        # Get top 3 gainers and losers
        top_gainers_indices = df_indices.head(5)
        top_losers_indices = df_indices.tail(5).iloc[::-1]

        print("\n" + "="*130)
        print("PHASE 2: SELECTING TOP 5 STOCKS FROM BEST & WORST PERFORMING INDICES")
        print("="*130 + "\n")

        gainer_stocks_details = {}
        loser_stocks_details = {}

        # Process top gainer indices
        print("📈 TOP 3 GAINER INDICES:\n")
        for idx, row in top_gainers_indices.iterrows():
            sector_name = row['Sector']
            print(f"  {sector_name} ({row['Change %']:+.2f}%)")

            if sector_name in all_stocks_by_index:
                stocks = all_stocks_by_index[sector_name]

                # Sort stocks by change % and get top 5
                stocks_df = pd.DataFrame(stocks).sort_values('Change %', ascending=False)
                top_5 = stocks_df.head(5).to_dict('records')

                gainer_stocks_details[sector_name] = {
                    'change_pct': row['Change %'],
                    'stocks': top_5,
                    'total_stocks': len(stocks)
                }
                print(f"      ✅ Top 5 selected from {len(stocks)} constituent stocks")
            else:
                print(f"      ⚠️ No constituent stock data available")

        # Process top loser indices
        print(f"\n📉 TOP 3 LOSER INDICES:\n")
        for idx, row in top_losers_indices.iterrows():
            sector_name = row['Sector']
            print(f"  {sector_name} ({row['Change %']:+.2f}%)")

            if sector_name in all_stocks_by_index:
                stocks = all_stocks_by_index[sector_name]

                # Sort stocks by change % and get top 5
                stocks_df = pd.DataFrame(stocks).sort_values('Change %', ascending=False)
                top_5 = stocks_df.head(5).to_dict('records')

                loser_stocks_details[sector_name] = {
                    'change_pct': row['Change %'],
                    'stocks': top_5,
                    'total_stocks': len(stocks)
                }
                print(f"      ✅ Top 5 selected from {len(stocks)} constituent stocks")
            else:
                print(f"      ⚠️ No constituent stock data available")

        # Display comprehensive results
        print("\n" + "="*130)
        print("=== 🚀 TOP GAINER INDICES WITH THEIR TOP 5 PERFORMING STOCKS ===")
        print("="*130)

        for index_name, details in gainer_stocks_details.items():
            print(f"\n{'='*100}")
            print(f"🚀 {index_name}")
            print(f"   Index Performance: {details['change_pct']:+.2f}% | Constituent Stocks: {details['total_stocks']}")
            print(f"{'='*100}")
            print(f"{'Rank':<6} {'Symbol':<15} {'Company':<40} {'LTP':<15} {'Change %':<10}")
            print("-" * 100)

            for rank, stock in enumerate(details['stocks'], 1):
                emoji = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
                print(f"{rank:<6} {stock['Symbol']:<15} {stock['Company'][:38]:<40} "
                      f"{stock['LTP']:>12,.2f}   {stock['Change %']:>8.2f}% {emoji}")

        print("\n" + "="*130)
        print("=== 📉 TOP LOSER INDICES WITH THEIR TOP 5 PERFORMING STOCKS ===")
        print("="*130)

        for index_name, details in loser_stocks_details.items():
            print(f"\n{'='*100}")
            print(f"📉 {index_name}")
            print(f"   Index Performance: {details['change_pct']:+.2f}% | Constituent Stocks: {details['total_stocks']}")
            print(f"{'='*100}")
            print(f"{'Rank':<6} {'Symbol':<15} {'Company':<40} {'LTP':<15} {'Change %':<10}")
            print("-" * 100)

            for rank, stock in enumerate(details['stocks'], 1):
                emoji = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
                print(f"{rank:<6} {stock['Symbol']:<15} {stock['Company'][:38]:<40} "
                      f"{stock['LTP']:>12,.2f}   {stock['Change %']:>8.2f}% {emoji}")

        # Save data
        timestamp = now.strftime("%Y%m%d_%H%M%S")

        # Save indices data
        csv_filename = f'nse_sectoral_{timestamp}.csv'
        df_indices.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"\n✅ Sectoral data saved to '{csv_filename}'")

        # Save stock details
        if gainer_stocks_details or loser_stocks_details:
            stocks_filename = f'nse_sectoral_top_stocks_{timestamp}.csv'

            all_stock_rows = []

            # Add gainer stocks
            for index_name, details in gainer_stocks_details.items():
                for rank, stock in enumerate(details['stocks'], 1):
                    all_stock_rows.append({
                        'Type': 'Top Gainer Index',
                        'Index': index_name,
                        'Index Change %': details['change_pct'],
                        'Total Stocks in Index': details['total_stocks'],
                        'Rank': rank,
                        'Symbol': stock['Symbol'],
                        'Company': stock['Company'],
                        'LTP': stock['LTP'],
                        'Stock Change %': stock['Change %']
                    })

            # Add loser stocks
            for index_name, details in loser_stocks_details.items():
                for rank, stock in enumerate(details['stocks'], 1):
                    all_stock_rows.append({
                        'Type': 'Top Loser Index',
                        'Index': index_name,
                        'Index Change %': details['change_pct'],
                        'Total Stocks in Index': details['total_stocks'],
                        'Rank': rank,
                        'Symbol': stock['Symbol'],
                        'Company': stock['Company'],
                        'LTP': stock['LTP'],
                        'Stock Change %': stock['Change %']
                    })

            if all_stock_rows:
                stocks_df = pd.DataFrame(all_stock_rows)
                stocks_df.to_csv(stocks_filename, index=False, encoding='utf-8')
                print(f"✅ Top stocks saved to '{stocks_filename}'")
                print(f"   Total stocks exported: {len(all_stock_rows)}")

        # Summary statistics
        print("\n" + "="*130)
        print("=== 📊 FINAL SUMMARY ===")
        print("="*130)
        print(f"✓ Sectoral indices analyzed: {len(df_indices)}")
        print(f"✓ Indices with constituent stocks: {len(all_stocks_by_index)}")
        print(f"✓ Top gainer indices with stocks: {len(gainer_stocks_details)}")
        print(f"✓ Top loser indices with stocks: {len(loser_stocks_details)}")

        total_stocks = sum(len(d['stocks']) for d in gainer_stocks_details.values())
        total_stocks += sum(len(d['stocks']) for d in loser_stocks_details.values())
        print(f"✓ Total top performing stocks identified: {total_stocks}")

        if df_indices['Change %'].mean() > 0:
            print(f"\n📈 Market Sentiment: POSITIVE (Avg: {df_indices['Change %'].mean():+.2f}%)")
        else:
            print(f"\n📉 Market Sentiment: NEGATIVE (Avg: {df_indices['Change %'].mean():+.2f}%)")

        return df_indices, gainer_stocks_details, loser_stocks_details

    if __name__ == "__main__":
        try:
            print("="*130)
            print("🎯 NSE SECTORAL INDICES WITH TOP STOCK PERFORMERS")
            print("   Smart Extraction - Direct from get_index_details() DataFrame")
            print("="*130 + "\n")

            result = main()

            if result:
                print("\n" + "="*130)
                print("✅ ANALYSIS COMPLETED SUCCESSFULLY!")
                print("="*130)
            else:
                print("\n⚠️ No data could be fetched. Please check your connection and try again.")

        except KeyboardInterrupt:
            print("\n\n⚠️ Interrupted by user")
        except Exception as e:
            print(f"\n\n❌ Unexpected Error: {str(e)}")
            import traceback
            traceback.print_exc()

Current time: 2025-09-30 13:07:08
Note: NSE market hours are 9:15 AM - 3:30 PM IST

🎯 NSE SECTORAL INDICES WITH TOP STOCK PERFORMERS
   Smart Extraction - Direct from get_index_details() DataFrame

PHASE 1: FETCHING SECTORAL INDICES DATA WITH CONSTITUENT STOCKS

[ 1/30] NIFTY INDIA CONSUMPTION             
      Analyzing DataFrame structure...
      Shape: (31, 16)
      Columns: ['symbol', 'lastUpdateTime', 'previousClose', 'open', 'dayHigh', 'dayLow', 'lastPrice', 'change', 'pChange', 'totalTradedVolume', 'totalTradedValue', 'nearWKH', 'nearWKL', 'perChange30d', 'perChange365d', 'ffmc']
      Detected columns: symbol=symbol, name=None, ltp=lastPrice, change%=pChange
✅  -0.28% | 30 stocks
[ 2/30] NIFTY FINANCIAL SERVICES            ✅   0.14% | 20 stocks
[ 3/30] NIFTY PHARMA                        ✅  -0.04% | 20 stocks
[ 4/30] NIFTY AUTO                          ✅   0.14% | 15 stocks
[ 5/30] NIFTY PRIVATE BANK                  ✅   0.26% | 10 stocks
[ 6/30] NIFTY COMMODITIES           

In [16]:
"""
Complete R.Fac Scanner with Bullish/Bearish Signal Detection
Matches TradeFinder methodology with proper activity and direction signals
"""

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

class RFacScanner:
    """
    Complete R.Fac Calculator with Bullish/Bearish Signal Detection
    R.Fac measures ACTIVITY (how much trading is happening)
    Signal tells you DIRECTION (bullish 🐂 or bearish 🐻)
    """

    def __init__(self, symbol):
        self.symbol = symbol
        self.data = None

    def fetch_data(self, days=100):
        """Fetch historical data from Yahoo Finance"""
        try:
            ticker = f"{self.symbol}.NS"  # Try NSE first
            self.data = yf.download(ticker, period=f"{days}d", progress=False)

            if len(self.data) == 0:
                ticker = f"{self.symbol}.BO"  # Try BSE
                self.data = yf.download(ticker, period=f"{days}d", progress=False)

            return len(self.data) > 0
        except Exception as e:
            print(f"Error fetching data for {self.symbol}: {e}")
            return False

    def calculate_rvol(self):
        """
        Calculate Relative Volume (RVOL) - 40% weight
        Uses CURRENT day volume vs 20-day average for intraday detection
        """
        if self.data is None or len(self.data) < 20:
            return 0, 0

        # Current volume
        current_volume = self.data['Volume'].iloc[-1]

        # Average volume (20-day)
        avg_volume_20d = self.data['Volume'].tail(20).mean()

        if avg_volume_20d == 0:
            return 0, 0

        # RVOL ratio
        rvol_ratio = current_volume / avg_volume_20d

        # Aggressive scoring to match TradeFinder
        if rvol_ratio >= 2.5:
            rvol_score = 100
        elif rvol_ratio >= 2.0:
            rvol_score = 90
        elif rvol_ratio >= 1.5:
            rvol_score = 75
        elif rvol_ratio >= 1.2:
            rvol_score = 50
        else:
            rvol_score = max((rvol_ratio - 1) * 50, 0)

        return rvol_score, rvol_ratio

    def calculate_momentum(self):
        """
        Calculate Price Momentum - 35% weight
        Uses ABSOLUTE values for activity, returns direction separately
        """
        if self.data is None or len(self.data) < 6:
            return 0, 0, 0, 0

        current_close = self.data['Close'].iloc[-1]
        close_1d_ago = self.data['Close'].iloc[-2]
        close_5d_ago = self.data['Close'].iloc[-6]

        # Calculate returns with DIRECTION
        returns_1d = ((current_close - close_1d_ago) / close_1d_ago) * 100
        returns_5d = ((current_close - close_5d_ago) / close_5d_ago) * 100

        # Calculate ABSOLUTE momentum for activity scoring
        abs_momentum_1d = abs(returns_1d)
        abs_momentum_5d = abs(returns_5d)

        # Combined absolute momentum (weighted)
        combined_abs_momentum = (abs_momentum_1d * 0.4) + (abs_momentum_5d * 0.6)

        # Score based on absolute movement (higher movement = higher score)
        momentum_score = min(combined_abs_momentum * 10, 100)

        # Directional momentum for signal
        directional_momentum = (returns_1d * 0.4) + (returns_5d * 0.6)

        return momentum_score, directional_momentum, returns_1d, returns_5d

    def calculate_rsi(self, period=14):
        """Calculate RSI - 15% weight"""
        if self.data is None or len(self.data) < period + 1:
            return 0, 0

        delta = self.data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        current_rsi = rsi.iloc[-1]

        # Score RSI based on range
        if 50 <= current_rsi <= 70:
            rsi_score = 100
        elif 40 <= current_rsi < 50:
            rsi_score = 70
        elif 70 < current_rsi <= 80:
            rsi_score = 80
        elif current_rsi > 80:
            rsi_score = 60  # Overbought
        elif current_rsi < 30:
            rsi_score = 40  # Oversold
        else:
            rsi_score = 50

        return rsi_score, current_rsi

    def calculate_trend(self):
        """Calculate Trend Strength - 10% weight"""
        if self.data is None or len(self.data) < 50:
            return 0, 0, 0

        ema_20 = self.data['Close'].ewm(span=20, adjust=False).mean()
        ema_50 = self.data['Close'].ewm(span=50, adjust=False).mean()

        current_close = self.data['Close'].iloc[-1]
        current_ema20 = ema_20.iloc[-1]
        current_ema50 = ema_50.iloc[-1]

        # Trend scoring
        if current_close > current_ema20 and current_close > current_ema50:
            trend_score = 100
        elif current_close > current_ema50:
            trend_score = 70
        elif current_close > current_ema20:
            trend_score = 50
        else:
            trend_score = 30

        return trend_score, current_ema20, current_ema50

    def calculate_rfac_with_signal(self):
        """
        Calculate complete R.Fac with Bullish/Bearish signal
        Returns: R.Fac value, signal, and detailed breakdown
        """
        if not self.fetch_data():
            return None

        # Calculate all components
        rvol_score, rvol_ratio = self.calculate_rvol()
        momentum_score, directional_momentum, ret_1d, ret_5d = self.calculate_momentum()
        rsi_score, rsi_value = self.calculate_rsi()
        trend_score, ema20, ema50 = self.calculate_trend()

        # Weighted composite score (for activity)
        composite_score = (
            rvol_score * 0.40 +
            momentum_score * 0.35 +
            rsi_score * 0.15 +
            trend_score * 0.10
        )

        # R.Fac value (0-10 scale)
        rfac = composite_score / 10

        # Determine signal (Bullish or Bearish)
        current_price = self.data['Close'].iloc[-1]
        prev_close = self.data['Close'].iloc[-2]
        price_change_pct = ((current_price - prev_close) / prev_close) * 100

        # Signal determination
        if directional_momentum > 0 and price_change_pct > 0:
            signal = "🐂 BULLISH"
            signal_strength = "STRONG" if rfac >= 6.0 else "MODERATE" if rfac >= 5.0 else "WEAK"
        elif directional_momentum < 0 and price_change_pct < 0:
            signal = "🐻 BEARISH"
            signal_strength = "STRONG" if rfac >= 6.0 else "MODERATE" if rfac >= 5.0 else "WEAK"
        else:
            signal = "⚠️ MIXED"
            signal_strength = "NEUTRAL"

        # Trading recommendation
        if rfac >= 5.0 and signal == "🐂 BULLISH":
            recommendation = "✅ BUY"
        elif rfac >= 5.0 and signal == "🐻 BEARISH":
            recommendation = "⚠️ SHORT / AVOID LONG"
        elif rfac >= 4.0:
            recommendation = "🟡 WATCH"
        else:
            recommendation = "🔴 AVOID"

        # Prepare results
        result = {
            'symbol': self.symbol,
            'rfac': round(rfac, 2),
            'rfac_change_pct': round(price_change_pct, 2),
            'signal': signal,
            'signal_strength': signal_strength,
            'recommendation': recommendation,
            'current_price': round(current_price, 2),
            'price_change': round(price_change_pct, 2),
            'volume': int(self.data['Volume'].iloc[-1]),
            'rvol_ratio': round(rvol_ratio, 2),
            'rvol_score': round(rvol_score, 2),
            'momentum_score': round(momentum_score, 2),
            'directional_momentum': round(directional_momentum, 2),
            'returns_1d': round(ret_1d, 2),
            'returns_5d': round(ret_5d, 2),
            'rsi': round(rsi_value, 2),
            'rsi_score': round(rsi_score, 2),
            'trend_score': round(trend_score, 2),
            'ema20': round(ema20, 2),
            'ema50': round(ema50, 2),
            'composite_score': round(composite_score, 2)
        }

        return result

    def print_detailed_report(self, result):
        """Print a detailed formatted report"""
        if result is None:
            print(f"❌ Could not fetch data for {self.symbol}")
            return

        print("=" * 90)
        print(f"R.FAC ANALYSIS: {result['symbol']}")
        print("=" * 90)
        print()

        # Main metrics
        print("📊 KEY METRICS:")
        print(f"   R.Fac:              {result['rfac']}  ({result['signal_strength']} Activity)")
        print(f"   Signal:             {result['signal']}")
        print(f"   Recommendation:     {result['recommendation']}")
        print(f"   Current Price:      ₹{result['current_price']}")
        print(f"   Price Change:       {result['price_change']:+.2f}%")
        print()

        # Component breakdown
        print("-" * 90)
        print("COMPONENT BREAKDOWN:")
        print("-" * 90)
        print(f"   1. RVOL (40% weight):")
        print(f"      - Current Volume:     {result['volume']:,}")
        print(f"      - RVOL Ratio:         {result['rvol_ratio']}x")
        print(f"      - RVOL Score:         {result['rvol_score']}/100")
        print(f"      - Status:             {'🔥 HIGH' if result['rvol_ratio'] >= 1.5 else '📊 Normal'}")
        print()

        print(f"   2. MOMENTUM (35% weight):")
        print(f"      - 1-Day Return:       {result['returns_1d']:+.2f}%")
        print(f"      - 5-Day Return:       {result['returns_5d']:+.2f}%")
        print(f"      - Directional:        {result['directional_momentum']:+.2f}%")
        print(f"      - Momentum Score:     {result['momentum_score']}/100")
        print(f"      - Status:             {'🚀 Strong' if abs(result['directional_momentum']) > 3 else '📈 Moderate'}")
        print()

        print(f"   3. RSI (15% weight):")
        print(f"      - RSI Value:          {result['rsi']:.2f}")
        print(f"      - RSI Score:          {result['rsi_score']}/100")
        print(f"      - Status:             ", end="")
        if result['rsi'] > 70:
            print("⚠️ Overbought")
        elif result['rsi'] < 30:
            print("⚠️ Oversold")
        elif 50 <= result['rsi'] <= 70:
            print("✅ Optimal")
        else:
            print("🟡 Neutral")
        print()

        print(f"   4. TREND (10% weight):")
        print(f"      - EMA(20):            ₹{result['ema20']}")
        print(f"      - EMA(50):            ₹{result['ema50']}")
        print(f"      - Trend Score:        {result['trend_score']}/100")
        if result['current_price'] > result['ema20'] and result['current_price'] > result['ema50']:
            print(f"      - Status:             ✅ Above both EMAs (Strong uptrend)")
        elif result['current_price'] > result['ema50']:
            print(f"      - Status:             🟡 Above EMA50 only")
        else:
            print(f"      - Status:             ❌ Below EMAs (Weak trend)")
        print()

        # Final composite
        print("-" * 90)
        print(f"COMPOSITE SCORE: {result['composite_score']}/100")
        print(f"R.FAC: {result['rfac']}/10")
        print("-" * 90)
        print()

        # Interpretation
        print("💡 INTERPRETATION:")
        if result['rfac'] >= 5.0:
            print(f"   ✅ HIGH ACTIVITY stock (R.Fac {result['rfac']} > 5.0)")
            if result['signal'] == "🐂 BULLISH":
                print(f"   ✅ BULLISH momentum ({result['directional_momentum']:+.2f}%)")
                print(f"   📈 Recommendation: {result['recommendation']}")
                print(f"   💰 Strong buying opportunity for momentum traders")
            elif result['signal'] == "🐻 BEARISH":
                print(f"   ⚠️ BEARISH momentum ({result['directional_momentum']:+.2f}%)")
                print(f"   📉 Recommendation: {result['recommendation']}")
                print(f"   ⚠️ Consider shorting or wait for reversal")
            else:
                print(f"   ⚠️ MIXED signals - conflicting momentum")
                print(f"   ⏸️ Best to wait for clearer direction")
        else:
            print(f"   ⚠️ MODERATE/LOW activity (R.Fac {result['rfac']} < 5.0)")
            print(f"   📊 Not ideal for momentum trading")
            print(f"   🔍 Better opportunities available in stocks with R.Fac > 5.0")
        print()
        print("=" * 90)
        print()


def scan_multiple_stocks(symbols, show_details=False):
    """
    Scan multiple stocks and return sorted by R.Fac
    """
    results = []

    print(f"Scanning {len(symbols)} stocks...\n")

    for symbol in symbols:
        print(f"Processing {symbol}...", end=" ")
        scanner = RFacScanner(symbol)
        result = scanner.calculate_rfac_with_signal()

        if result:
            results.append(result)
            print(f"✓ R.Fac: {result['rfac']} {result['signal']}")
        else:
            print("✗ Failed")

    if not results:
        print("\n❌ No results obtained")
        return None

    # Create DataFrame and sort by R.Fac
    df = pd.DataFrame(results)
    df = df.sort_values('rfac', ascending=False)

    # Display summary table
    print("\n" + "=" * 120)
    print("R.FAC SCANNER RESULTS - SORTED BY ACTIVITY")
    print("=" * 120)

    summary_df = df[['symbol', 'rfac', 'signal', 'recommendation', 'current_price',
                     'price_change', 'rvol_ratio', 'directional_momentum']].copy()
    summary_df.columns = ['Symbol', 'R.Fac', 'Signal', 'Action', 'Price', 'Change %', 'RVOL', 'Momentum %']

    print(summary_df.to_string(index=False))
    print("=" * 120)
    print()

    # Show detailed reports if requested
    if show_details:
        print("\n" + "=" * 120)
        print("DETAILED REPORTS FOR TOP STOCKS")
        print("=" * 120)
        print()

        for i, row in df.head(5).iterrows():
            scanner = RFacScanner(row['symbol'])
            scanner.data = scanner.fetch_data()
            scanner.print_detailed_report(row)

    return df


def filter_by_criteria(df, min_rfac=5.0, signal_type=None):
    """
    Filter results by criteria
    signal_type: 'BULLISH', 'BEARISH', or None for all
    """
    filtered = df[df['rfac'] >= min_rfac].copy()

    if signal_type:
        signal_map = {'BULLISH': '🐂 BULLISH', 'BEARISH': '🐻 BEARISH'}
        if signal_type in signal_map:
            filtered = filtered[filtered['signal'] == signal_map[signal_type]]

    return filtered


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":

    print("\n")
    print("=" * 120)
    print("R.FAC SCANNER WITH BULLISH/BEARISH SIGNAL DETECTION")
    print("=" * 120)
    print()

    # ========================================================================
    # EXAMPLE 1: Single Stock Detailed Analysis
    # ========================================================================

    print("EXAMPLE 1: DETAILED ANALYSIS FOR SINGLE STOCK")
    print("-" * 120)
    print()

    stock = "KPITTECH"
    scanner = RFacScanner(stock)
    result = scanner.calculate_rfac_with_signal()
    scanner.print_detailed_report(result)

    # ========================================================================
    # EXAMPLE 2: Batch Scanning Multiple Stocks
    # ========================================================================

    print("\n\n")
    print("EXAMPLE 2: BATCH SCANNING MULTIPLE STOCKS")
    print("-" * 120)
    print()

    # List of stocks to scan
    stock_list = [
        'RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK',
        'BHARTIARTL', 'ITC', 'LT', 'AXISBANK', 'KOTAKBANK',
        'SBIN', 'TATAMOTORS', 'BAJFINANCE', 'MARUTI', 'HINDUNILVR',
        'TATASTEEL', 'WIPRO', 'ONGC', 'NTPC', 'POWERGRID'
    ]

    # Scan all stocks
    results_df = scan_multiple_stocks(stock_list, show_details=False)

    # ========================================================================
    # EXAMPLE 3: Filter High Activity Bullish Stocks
    # ========================================================================

    if results_df is not None:
        print("\n" + "=" * 120)
        print("FILTERED RESULTS: HIGH ACTIVITY BULLISH STOCKS (R.Fac > 5.0)")
        print("=" * 120)
        print()

        bullish_stocks = filter_by_criteria(results_df, min_rfac=5.0, signal_type='BULLISH')

        if len(bullish_stocks) > 0:
            print(f"Found {len(bullish_stocks)} bullish stocks with high activity:\n")
            display_cols = ['symbol', 'rfac', 'signal', 'recommendation', 'price_change', 'rvol_ratio']
            print(bullish_stocks[display_cols].to_string(index=False))
            print("\n✅ These are your best BUY opportunities based on R.Fac analysis!")
        else:
            print("❌ No bullish stocks with R.Fac > 5.0 found in current scan")

        print()

        # ====================================================================
        # EXAMPLE 4: Filter High Activity Bearish Stocks (Short Opportunities)
        # ====================================================================

        print("=" * 120)
        print("FILTERED RESULTS: HIGH ACTIVITY BEARISH STOCKS (R.Fac > 5.0)")
        print("=" * 120)
        print()

        bearish_stocks = filter_by_criteria(results_df, min_rfac=5.0, signal_type='BEARISH')

        if len(bearish_stocks) > 0:
            print(f"Found {len(bearish_stocks)} bearish stocks with high activity:\n")
            display_cols = ['symbol', 'rfac', 'signal', 'recommendation', 'price_change', 'rvol_ratio']
            print(bearish_stocks[display_cols].to_string(index=False))
            print("\n⚠️ These are potential SHORT opportunities or stocks to AVOID for long positions!")
        else:
            print("❌ No bearish stocks with R.Fac > 5.0 found in current scan")

        print()
        print("=" * 120)
        print()

    # ========================================================================
    # EXPORT TO CSV
    # ========================================================================

    if results_df is not None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"rfac_scan_{timestamp}.csv"
        results_df.to_csv(filename, index=False)
        print(f"✅ Results exported to: {filename}")
        print()

    print("=" * 120)
    print("SCAN COMPLETE!")
    print("=" * 120)



R.FAC SCANNER WITH BULLISH/BEARISH SIGNAL DETECTION

EXAMPLE 1: DETAILED ANALYSIS FOR SINGLE STOCK
------------------------------------------------------------------------------------------------------------------------



C:\Users\Sanjay.Ladhe\AppData\Local\Temp\ipykernel_15696\2565802275.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.data = yf.download(ticker, period=f"{days}d", progress=False)
Failed to get ticker 'KPITTECH.NS' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

1 Failed download:
['KPITTECH.NS']: CertificateVerifyError('Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
C:\Users\Sanjay.Ladhe\AppData\Local\Temp\ipykernel_15696\2565802275.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.data = yf.download(ticker, period=f"{days}d", progress=False)
Failed to get ticker 'KPITTECH.BO' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get 

❌ Could not fetch data for KPITTECH



EXAMPLE 2: BATCH SCANNING MULTIPLE STOCKS
------------------------------------------------------------------------------------------------------------------------

Scanning 20 stocks...

Processing RELIANCE... 

Failed to get ticker 'RELIANCE.NS' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

1 Failed download:
['RELIANCE.NS']: CertificateVerifyError('Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
C:\Users\Sanjay.Ladhe\AppData\Local\Temp\ipykernel_15696\2565802275.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.data = yf.download(ticker, period=f"{days}d", progress=False)
Failed to get ticker 'RELIANCE.BO' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

1 Failed download:
['RELIANCE.BO']: CertificateVerifyError('Failed to perform, curl: (60) SSL certificate problem: unable

✗ Failed
Processing TCS... 

...

✗ Failed
Processing NTPC... 


1 Failed download:
['NTPC.NS']: CertificateVerifyError('Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
C:\Users\Sanjay.Ladhe\AppData\Local\Temp\ipykernel_15696\2565802275.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.data = yf.download(ticker, period=f"{days}d", progress=False)
Failed to get ticker 'NTPC.BO' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

1 Failed download:
['NTPC.BO']: CertificateVerifyError('Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
C:\Users\Sanjay.Ladhe\AppData\Local\Temp\ipykernel_15696\2565802275.py:26: FutureWarning: YF.download() has change

✗ Failed
Processing POWERGRID... 


1 Failed download:
['POWERGRID.NS']: CertificateVerifyError('Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
C:\Users\Sanjay.Ladhe\AppData\Local\Temp\ipykernel_15696\2565802275.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.data = yf.download(ticker, period=f"{days}d", progress=False)
Failed to get ticker 'POWERGRID.BO' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

1 Failed download:
['POWERGRID.BO']: CertificateVerifyError('Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


✗ Failed

❌ No results obtained
SCAN COMPLETE!
